In [1]:
using JuMP
using HiGHS
import Random 
using Plots 

In [2]:
rng = Random.MersenneTwister(1234);

## One fake house which everyone hates to reflect an empty allocation  
function generate_prefs(n,h)
    prefs = zeros(h+1, n)
    for i=1:n 
        prefs[1:h, i] = Random.shuffle(rng, 1:h)
    end 
    prefs[h+1, :] .= h+1 
    return prefs 
end 


function generate_caps(h, m_cap)
    caps = zeros(h+1)
    caps[1:h] = rand(1:m_cap, h) 
    caps[h+1] = Inf 
    return caps 
end 


function lottery_round_house!(prefs, all, house, cap, used)
    n = size(prefs)[2]
    n_changed = 0 
    lottery = []
    for pp=1:n
        h = all[pp]
        #my_score = findfirst( prefs[:, pp] .== h )
        #upgrade =  findfirst( prefs[:, pp] .== house)
        my_score = prefs[h, pp] 
        upgrade = prefs[house, pp]
        if my_score > upgrade 
            push!(lottery, pp) 
        end 
    end 
    Random.shuffle!(lottery)
    while used[house] < cap[house] && length(lottery) > 0 
        lucky = pop!(lottery) 
        old = all[lucky]
        used[old] -= 1
        all[lucky] = house 
        used[house] += 1 
        n_changed += 1 
    end 
    return n_changed 
end 

function lottery_round!(prefs, all, cap, used)
    n_changed = 0 
    n_houses = size(prefs)[1]
    for h=1:n_houses
        n_changed += lottery_round_house!(prefs, all, h, cap, used)
    end 
    return n_changed
end 

function lottery!(prefs, all, cap, used)
    n_changed = 1 
    total = 0 

    while n_changed != 0 
        n_changed = lottery_round!(prefs, all, cap, used)
        total += 1
    end 
    return total
end 


function score(prefs, all)
    total = 0 
    for i=1:n 
        total += prefs[all[i], i] 
    end 
    return total 
end 

function score_numeric(prefs_num, prefs, all) 
    total = 0 
    n = size(prefs)[2]
    
    for i=1:n 
        place =  findfirst( prefs[:, i] .== all[i]) 
        total += prefs_num[place, i]
    end  
    return total 
end    





score_numeric (generic function with 1 method)

In [5]:
n = 1000
houses = 100
prefs = generate_prefs(n, houses)
caps = generate_caps(houses, 30)
all = (houses + 1) * ones(Int32, n)
used = zeros(houses+1)
used[houses+1] = n  


util = zeros(houses)
usage = zeros(houses)
orig_caps = copy(caps)
all_placed = zeros(houses)
n_rounds = 1 

1

In [111]:

for i=1:100
    caps[1:i] .= 0 
    for k=1:n_rounds
        lottery!(prefs, all, caps, used)
        util[i] = score_numeric(prefs_num, prefs, all)/n  
        usage[i] = length(filter(n -> n != houses+1, all)) /  sum(caps[1:houses])
        all_placed[i] = 1  - any( all .== houses+1  )
        caps[1:5] = orig_caps[1:5] 
        #lottery!(prefs, all, caps, used)
        #util[i] += score_numeric(prefs_num, prefs, all)/n 
        global caps = copy(orig_caps)
        all .= (houses + 1) 
        used .= 0 
        used[houses+1] = n 
    end 
    util[i] /= n_rounds
end 



In [123]:
prefs

1001×10000 Matrix{Float64}:
  461.0   719.0   435.0   772.0    93.0  …   534.0     1.0   766.0   430.0
  142.0   620.0   766.0   648.0   784.0        1.0    52.0   465.0   202.0
  812.0    51.0   878.0   844.0   309.0      472.0   282.0   155.0   668.0
  155.0   609.0   158.0   164.0   344.0       39.0    88.0   150.0   624.0
  819.0   767.0   650.0   719.0   467.0      994.0   493.0   680.0   284.0
   56.0   311.0    45.0   174.0   308.0  …   504.0    96.0   223.0   609.0
  986.0   876.0   769.0   707.0   187.0      787.0   537.0   895.0   205.0
  647.0   457.0   662.0   599.0   639.0      547.0   112.0   672.0   385.0
  531.0   371.0   813.0   686.0   737.0      263.0   676.0   185.0   994.0
  331.0   745.0   277.0   352.0   617.0      570.0   637.0    63.0   420.0
    ⋮                                    ⋱                          
  768.0   934.0   856.0   198.0   291.0      606.0   295.0   503.0    74.0
  975.0   455.0   356.0   786.0    76.0      566.0   968.0   111.0   219.0
  1

In [124]:
plot(1:100, util, label="Marginal utility of 5 units")
plot!(twinx(), usage, color="red", label="usage rate")

title!("Housing capacity vs Utility(Pop=1000)")
xlabel!("Housing Supply")
ylabel!("Average Utility")


In [33]:
model = Model(HiGHS.Optimizer)
all2 = zeros(houses+1, n )
@variable(model, all2[1:houses+1, 1:n], Bin)
#@variable(model, 0 <= all2[1:houses+1, 1:n] <= 1)
@objective(model, Min, sum( all2 .* prefs)  )
for i=1:n
    @constraint(model,  sum(all2[1:houses+1, i]) == 1)
end 
for i=1:n 
    @constraint(model, all2[1:houses, i]' * prefs[1:houses, i] <= prefs[all[i], i] )
end 
for j=1:houses
    @constraint(model,  sum(all2[j, 1:n]) <= caps[j] )
end 


In [34]:
display(model)

A JuMP Model
Minimization problem with:
Variables: 101000
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1000 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 1100 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 101000 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: all2

In [35]:
optimize!(model)

Running HiGHS 1.3.0 [date: 1970-01-01, git hash: e5004072b-dirty]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
2082 rows, 47516 cols, 140566 nonzeros
2043 rows, 47512 cols, 140502 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   2043 rows
   47512 cols (47508 binary, 4 integer, 0 implied int., 0 continuous)
   140502 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   18              inf                  inf        0      0      0         0     0.5s
 T       0       0         0   0.00%   18              1151              98.44%        0      0      0      1060     0.6s

Solving report
  Status            Optimal
  Primal bound      1151
  Dual bound        1151
  Gap               0% (toleran

In [6]:
all .= houses+1
used .= 0
caps[50:houses] .= 0 
lottery!(prefs, all, caps, used)
score(prefs, all)

46709.0

In [7]:
score(prefs,all)/n

46.709

In [8]:
caps .= orig_caps 


101-element Vector{Float64}:
 27.0
 11.0
 28.0
  3.0
  5.0
 17.0
 14.0
 25.0
  7.0
 27.0
  ⋮
 12.0
 22.0
 11.0
 20.0
 29.0
  7.0
 15.0
 17.0
 Inf

In [24]:
value.(all2[1:houses, 1])' * prefs[1:houses, 1]

1.0

In [32]:
prefs[all[2],2]

101.0